In [1]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
import openpyxl
import akshare as ak
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 5000
pd.options.display.float_format = lambda x: '%.5f' % x
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.sans-serif'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False
%config InlineBackend.figure_format='svg'
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

In [2]:
def excess_profit(code):
    # 去除单引号
    code = code.strip("'")
    
    try:
        # 获取股票数据
        price = ak.stock_zh_a_hist(symbol=code, period='daily', adjust='hfq', 
                                  start_date='20221201', end_date='20250101')
        
        # 检查是否获取到数据
        if price.empty:
            print(f"警告: 股票 {code} 没有历史数据")
            return 0.0, 0.0, 0.0

        
        # 检查关键列是否存在
        required_columns = ['日期', '开盘', '收盘', '最高', '最低']
        missing_columns = [col for col in required_columns if col not in price.columns]
        
        if missing_columns:
            print(f"错误: 股票 {code} 缺少必要的列: {missing_columns}")
            return 0.0, 0.0, 0.0
        
        # 数据预处理
        price = price.dropna()
        
        price['EMA12']=price['收盘'].ewm(span=12,adjust=False).mean().round(2)
        price['EMA26']=price['收盘'].ewm(span=26,adjust=False).mean().round(2)
        price['DIF']=(price['EMA12']-price['EMA26']).round(2)
        price['DEA']=price['DIF'].ewm(span=9,adjust=False).mean().round(2)
        price['MACD']=((price['DIF']-price['DEA'])*2).round(2)
        

        # 检查处理后的数据是否足够
        if len(price) < 100:
            print(f"错误: 股票 {code} 数据不足，无法进行回测")
            return 0.0, 0.0, 0.0
        
        # 初始化策略参数
        initial_cash = 10000000.0
        tax = 0.00025
        
        # 创建结果DataFrame
        df = pd.DataFrame(columns=['日期', '开盘', '收盘','DIF','DEA','MACD', '可买', '持仓', '交易信号', '总市值', '现金', '策略收益率', '股票收益率'],
                          index=range(len(price)))
        
        # 复制数据
        df[['日期', '开盘', '收盘','DIF','DEA','MACD']] = price[['日期', '开盘', '收盘','DIF','DEA','MACD']]
        df=df.drop(df.index[:22]).reset_index(drop=True)
        # 初始化列
        df['持仓'] = 0
        df['总市值'] = 0.0
        df['现金'] = initial_cash
        df['策略收益率'] = 0.0
        df['股票收益率'] = 0.0
        df['交易信号'] = 0
        df['总资产'] = initial_cash
        df['可买'] = 0

        # 执行策略回测
        for i in range(1, len(df)):
            # 计算可买股数
            df.loc[i, '可买'] = (((df.loc[i-1, '现金'] / df.loc[i, '开盘'])*(1-tax)) // 100) * 100
            
            # 生成交易信号
            if df.loc[i,'DIF']>=df.loc[i,'DEA'] and df.loc[i-1,'DIF']<df.loc[i-1,'DEA']:
                df.loc[i,'交易信号']=1
            if df.loc[i,'DIF']<=df.loc[i,'DEA'] and df.loc[i-1,'DIF']>df.loc[i-1,'DEA']:
                df.loc[i,'交易信号']=-1

                
            # 计算交易成本
            buy = 0
            sell = 0
            
            if i < len(df):
                buy = max(df.loc[i, '开盘'] * df.loc[i, '可买'] * tax, 5) + df.loc[i, '开盘'] * df.loc[i, '可买']
                
            if i in range(len(df)) and df.loc[i-1, '持仓'] > 0:
                sell = df.loc[i, '开盘'] * df.loc[i-1, '持仓'] - max(df.loc[i, '开盘'] * df.loc[i-1, '持仓'] * tax, 5)
                
            # 执行交易
            if df.loc[i-1, '交易信号'] == 1 and df.loc[i-1, '现金'] >= buy and df.loc[i-1, '持仓'] == 0 and i!=len(df)-1:
                df.loc[i, '现金'] = df.loc[i-1, '现金'] - buy
                df.loc[i, '持仓'] = df.loc[i-1, '持仓'] + df.loc[i, '可买']
            elif df.loc[i-1, '交易信号'] == -1 and df.loc[i-1, '持仓'] > 0:
                df.loc[i, '现金'] = df.loc[i-1, '现金'] + sell
                df.loc[i, '持仓'] = 0
            else:
                df.loc[i, '现金'] = df.loc[i-1, '现金']
                df.loc[i, '持仓'] = df.loc[i-1, '持仓']
                
            # 计算总资产和收益率
            df.loc[i, '总市值'] = df.loc[i, '持仓'] * df.loc[i, '收盘']
            df.loc[i, '总资产'] = df.loc[i, '总市值'] + df.loc[i, '现金']
            df.loc[i, '策略收益率'] = (df.loc[i, '总资产'] / initial_cash) - 1
            df.loc[i, '股票收益率'] = (df.loc[i, '收盘'] / df.loc[0, '收盘']) - 1
        
        # 计算最终结果
        if len(df) > 0:
            excess = df.loc[len(df)-1, '策略收益率'] - df.loc[len(df)-1, '股票收益率']
            strategy = df.loc[len(df)-1, '策略收益率']
            stock = df.loc[len(df)-1, '股票收益率']
            
            # 结果检测
            if not all([isinstance(x, (int, float)) for x in [excess, strategy, stock]]):
                print(f"错误: 股票 {code} 的收益率计算结果包含非数值类型")
                return 0.0, 0.0, 0.0
                
            if any(pd.isna([excess, strategy, stock])):
                print(f"错误: 股票 {code} 的收益率计算结果包含NaN值")
                return 0.0, 0.0, 0.0
                
            if any([abs(x) > 10 for x in [excess, strategy, stock]]):
                print(f"警告: 股票 {code} 的收益率计算结果异常大: "
                      f"超额收益率={excess:.4f}, 策略收益率={strategy:.4f}, 股票收益率={stock:.4f}")
                
            # 打印最终结果
            print(f"股票 {code} 策略回测完成: "
                  f"超额收益率={excess:.4f}, 策略收益率={strategy:.4f}, 股票收益率={stock:.4f}")
            
            return excess, strategy, stock
        else:
            print(f"错误: 股票 {code} 的回测结果DataFrame为空")
            return 0.0, 0.0, 0.0
            
    except Exception as e:
        print(f"处理股票 {code} 时出错: {e}")
        return 0.0, 0.0, 0.0

In [3]:
# 获取A股实时数据
df_excess = ak.stock_zh_a_spot()

# 移除代码中的前缀
df_excess['代码'] = df_excess['代码'].str[2:]
# 筛选沪深市股票
df_excess = df_excess[~df_excess['代码'].str.startswith(('4','8','9'))]
df_excess = df_excess[~df_excess['名称'].str.startswith(('ST','*ST'))]
# 重置索引
df_excess = df_excess.reset_index(drop=True)

# 删除不需要的列
columns_to_drop = ['最新价', '涨跌额', '涨跌幅', '买入', '卖出', '昨收', '今开', '最高', '最低', '成交量', '成交额', '时间戳']
df_excess = df_excess.drop(columns=[col for col in columns_to_drop if col in df_excess.columns])

# 应用策略并填充结果
def calculate_returns(row):
    code = row['代码']
    return pd.Series(excess_profit(code), index=['超额收益率', '策略收益率', '股票收益率'])

# 先创建新的列，再应用函数
df_excess[['超额收益率', '策略收益率', '股票收益率']] = df_excess.apply(calculate_returns, axis=1)

# 过滤掉三个收益率都为0的行
df_excess = df_excess[~((df_excess['超额收益率'] == 0) & 
                        (df_excess['策略收益率'] == 0) & 
                        (df_excess['股票收益率'] == 0))]

# 删除包含缺失值的行
df_excess = df_excess.dropna()

# 按超额收益率降序排列
#df_excess = df_excess.sort_values(by='超额收益率', ascending=False)
#df_excess=df_excess.reset_index(drop=True)

# 将收益率转换为百分比格式
#df_excess[['超额收益率', '策略收益率', '股票收益率']] = df_excess[['超额收益率', '策略收益率', '股票收益率']].apply(lambda x: x * 100).applymap("{:.2f}%".format)

# 保存结果到Excel文件
df_excess.to_excel(r'E:/jupyter/excel/MACD策略回测.xlsx')
df_excess

Please wait for a moment:   0%|          | 0/68 [00:00<?, ?it/s]

股票 600000 策略回测完成: 超额收益率=-0.0616, 策略收益率=0.2543, 股票收益率=0.3160
股票 600004 策略回测完成: 超额收益率=0.1501, 策略收益率=-0.1281, 股票收益率=-0.2782
股票 600006 策略回测完成: 超额收益率=-0.1193, 策略收益率=0.1247, 股票收益率=0.2440
股票 600007 策略回测完成: 超额收益率=-0.0114, 策略收益率=0.4793, 股票收益率=0.4907
股票 600008 策略回测完成: 超额收益率=-0.1821, 策略收益率=-0.0071, 股票收益率=0.1749
股票 600009 策略回测完成: 超额收益率=0.1567, 策略收益率=-0.1959, 股票收益率=-0.3527
股票 600010 策略回测完成: 超额收益率=-0.1499, 策略收益率=-0.1812, 股票收益率=-0.0313
股票 600011 策略回测完成: 超额收益率=-0.1289, 策略收益率=-0.2049, 股票收益率=-0.0760
股票 600012 策略回测完成: 超额收益率=-0.6503, 策略收益率=0.3333, 股票收益率=0.9836
股票 600015 策略回测完成: 超额收益率=-0.3143, 策略收益率=0.1051, 股票收益率=0.4194
股票 600016 策略回测完成: 超额收益率=-0.1463, 策略收益率=0.0561, 股票收益率=0.2024
股票 600017 策略回测完成: 超额收益率=-0.0836, 策略收益率=0.0672, 股票收益率=0.1508
股票 600018 策略回测完成: 超额收益率=-0.1029, 策略收益率=0.0430, 股票收益率=0.1460
股票 600019 策略回测完成: 超额收益率=-0.1328, 策略收益率=0.0381, 股票收益率=0.1709
股票 600020 策略回测完成: 超额收益率=-0.3270, 策略收益率=0.0012, 股票收益率=0.3282
股票 600021 策略回测完成: 超额收益率=0.0899, 策略收益率=-0.0005, 股票收益率=-0.0904
股票 600022 策略回测完成: 超额收益率=-0.1068,

股票 600187 策略回测完成: 超额收益率=0.6675, 策略收益率=0.5924, 股票收益率=-0.0750
股票 600188 策略回测完成: 超额收益率=-0.0463, 策略收益率=-0.0258, 股票收益率=0.0204
股票 600189 策略回测完成: 超额收益率=-0.1439, 策略收益率=-0.0808, 股票收益率=0.0631
股票 600191 策略回测完成: 超额收益率=-0.4874, 策略收益率=-0.0432, 股票收益率=0.4442
股票 600192 策略回测完成: 超额收益率=-0.1485, 策略收益率=-0.1627, 股票收益率=-0.0142
股票 600195 策略回测完成: 超额收益率=0.1039, 策略收益率=-0.2399, 股票收益率=-0.3438
股票 600196 策略回测完成: 超额收益率=0.0894, 策略收益率=-0.1664, 股票收益率=-0.2559
股票 600197 策略回测完成: 超额收益率=0.1758, 策略收益率=-0.0328, 股票收益率=-0.2085
股票 600198 策略回测完成: 超额收益率=-1.7249, 策略收益率=-0.1492, 股票收益率=1.5756
股票 600199 策略回测完成: 超额收益率=0.0891, 策略收益率=-0.4089, 股票收益率=-0.4979
股票 600201 策略回测完成: 超额收益率=0.2657, 策略收益率=0.0666, 股票收益率=-0.1991
股票 600202 策略回测完成: 超额收益率=0.3705, 策略收益率=0.2495, 股票收益率=-0.1211
股票 600203 策略回测完成: 超额收益率=-0.4938, 策略收益率=-0.0081, 股票收益率=0.4857
股票 600206 策略回测完成: 超额收益率=0.6198, 策略收益率=0.8264, 股票收益率=0.2066
股票 600207 策略回测完成: 超额收益率=0.5920, 策略收益率=0.3516, 股票收益率=-0.2404
股票 600208 策略回测完成: 超额收益率=-0.1538, 策略收益率=-0.0283, 股票收益率=0.1255
股票 600210 策略回测完成: 超额收益率=-0.09

股票 600382 策略回测完成: 超额收益率=-0.1012, 策略收益率=0.0102, 股票收益率=0.1114
股票 600383 策略回测完成: 超额收益率=0.3078, 策略收益率=-0.0549, 股票收益率=-0.3627
股票 600386 策略回测完成: 超额收益率=0.1439, 策略收益率=0.2281, 股票收益率=0.0842
股票 600388 策略回测完成: 超额收益率=0.0370, 策略收益率=-0.0735, 股票收益率=-0.1105
股票 600389 策略回测完成: 超额收益率=0.3248, 策略收益率=-0.1170, 股票收益率=-0.4418
股票 600390 策略回测完成: 超额收益率=-0.0771, 策略收益率=0.1816, 股票收益率=0.2587
股票 600391 策略回测完成: 超额收益率=0.0015, 策略收益率=0.0844, 股票收益率=0.0829
股票 600392 策略回测完成: 超额收益率=0.1272, 策略收益率=-0.1456, 股票收益率=-0.2728
股票 600395 策略回测完成: 超额收益率=-0.0455, 策略收益率=-0.1274, 股票收益率=-0.0819
股票 600396 策略回测完成: 超额收益率=0.0027, 策略收益率=-0.0368, 股票收益率=-0.0396
股票 600397 策略回测完成: 超额收益率=-0.0224, 策略收益率=0.2691, 股票收益率=0.2915
股票 600398 策略回测完成: 超额收益率=-0.1418, 策略收益率=0.2627, 股票收益率=0.4045
股票 600399 策略回测完成: 超额收益率=0.2628, 策略收益率=-0.3425, 股票收益率=-0.6053
股票 600400 策略回测完成: 超额收益率=0.1772, 策略收益率=-0.1702, 股票收益率=-0.3474
股票 600403 策略回测完成: 超额收益率=0.0414, 策略收益率=-0.1761, 股票收益率=-0.2175
股票 600405 策略回测完成: 超额收益率=-0.1391, 策略收益率=-0.1949, 股票收益率=-0.0558
股票 600406 策略回测完成: 超额收益率=-0.256

股票 600579 策略回测完成: 超额收益率=0.2422, 策略收益率=1.1333, 股票收益率=0.8911
股票 600580 策略回测完成: 超额收益率=-0.2048, 策略收益率=0.1345, 股票收益率=0.3393
股票 600581 策略回测完成: 超额收益率=-0.0318, 策略收益率=-0.2335, 股票收益率=-0.2017
股票 600582 策略回测完成: 超额收益率=-0.2276, 策略收益率=-0.0112, 股票收益率=0.2164
股票 600583 策略回测完成: 超额收益率=-0.2062, 策略收益率=-0.2626, 股票收益率=-0.0563
股票 600584 策略回测完成: 超额收益率=-0.3069, 策略收益率=0.4560, 股票收益率=0.7630
股票 600585 策略回测完成: 超额收益率=0.1594, 策略收益率=0.1223, 股票收益率=-0.0370
股票 600586 策略回测完成: 超额收益率=0.1730, 策略收益率=-0.1759, 股票收益率=-0.3489
股票 600587 策略回测完成: 超额收益率=0.0595, 策略收益率=0.0644, 股票收益率=0.0049
股票 600588 策略回测完成: 超额收益率=0.3941, 策略收益率=-0.1365, 股票收益率=-0.5306
股票 600589 策略回测完成: 超额收益率=-0.9112, 策略收益率=0.4422, 股票收益率=1.3534
股票 600590 策略回测完成: 超额收益率=-0.0394, 策略收益率=-0.1380, 股票收益率=-0.0986
股票 600592 策略回测完成: 超额收益率=-0.1422, 策略收益率=0.1781, 股票收益率=0.3204
股票 600593 策略回测完成: 超额收益率=0.3600, 策略收益率=1.6022, 股票收益率=1.2421
股票 600594 策略回测完成: 超额收益率=0.0846, 策略收益率=-0.2219, 股票收益率=-0.3065
股票 600595 策略回测完成: 超额收益率=0.4442, 策略收益率=0.3181, 股票收益率=-0.1261
股票 600596 策略回测完成: 超额收益率=0.0398, 策

...

股票 301167 策略回测完成: 超额收益率=-0.2027, 策略收益率=-0.4027, 股票收益率=-0.2000
股票 301168 策略回测完成: 超额收益率=0.4127, 策略收益率=-0.2202, 股票收益率=-0.6328
股票 301169 策略回测完成: 超额收益率=1.0816, 策略收益率=1.3040, 股票收益率=0.2224
股票 301170 策略回测完成: 超额收益率=-0.0618, 策略收益率=-0.4344, 股票收益率=-0.3726
股票 301171 策略回测完成: 超额收益率=-0.6978, 策略收益率=-0.2269, 股票收益率=0.4710
股票 301172 策略回测完成: 超额收益率=0.3054, 策略收益率=-0.0478, 股票收益率=-0.3532
警告: 股票 301173 没有历史数据
股票 301175 策略回测完成: 超额收益率=-0.1227, 策略收益率=-0.2259, 股票收益率=-0.1032
股票 301176 策略回测完成: 超额收益率=0.2616, 策略收益率=0.1954, 股票收益率=-0.0662
股票 301177 策略回测完成: 超额收益率=0.0792, 策略收益率=-0.5168, 股票收益率=-0.5960
股票 301178 策略回测完成: 超额收益率=0.1665, 策略收益率=0.2702, 股票收益率=0.1038
股票 301179 策略回测完成: 超额收益率=-0.1716, 策略收益率=0.0572, 股票收益率=0.2288
股票 301180 策略回测完成: 超额收益率=-0.0814, 策略收益率=-0.1669, 股票收益率=-0.0856
股票 301181 策略回测完成: 超额收益率=0.0620, 策略收益率=-0.1594, 股票收益率=-0.2214
股票 301182 策略回测完成: 超额收益率=-0.1933, 策略收益率=0.6195, 股票收益率=0.8129
股票 301183 策略回测完成: 超额收益率=-0.2659, 策略收益率=1.0943, 股票收益率=1.3602
股票 301185 策略回测完成: 超额收益率=-0.2869, 策略收益率=-0.1861, 股票收益率=0.1008
股票 3011

股票 301329 策略回测完成: 超额收益率=-0.1784, 策略收益率=-0.3817, 股票收益率=-0.2033
股票 301330 策略回测完成: 超额收益率=-0.3059, 策略收益率=-0.2349, 股票收益率=0.0710
股票 301331 策略回测完成: 超额收益率=0.0877, 策略收益率=-0.0260, 股票收益率=-0.1136
股票 301332 策略回测完成: 超额收益率=0.2500, 策略收益率=-0.0302, 股票收益率=-0.2802
股票 301333 策略回测完成: 超额收益率=0.4724, 策略收益率=0.3748, 股票收益率=-0.0976
股票 301335 策略回测完成: 超额收益率=-0.1641, 策略收益率=-0.2572, 股票收益率=-0.0931
股票 301336 策略回测完成: 超额收益率=-0.2679, 策略收益率=-0.0413, 股票收益率=0.2266
股票 301337 策略回测完成: 超额收益率=0.1362, 策略收益率=-0.0015, 股票收益率=-0.1376
股票 301338 策略回测完成: 超额收益率=0.3006, 策略收益率=0.1415, 股票收益率=-0.1591
股票 301339 策略回测完成: 超额收益率=-0.4424, 策略收益率=-0.2361, 股票收益率=0.2064
股票 301345 策略回测完成: 超额收益率=-0.2351, 策略收益率=0.2120, 股票收益率=0.4471
股票 301348 策略回测完成: 超额收益率=0.7330, 策略收益率=0.2008, 股票收益率=-0.5322
股票 301349 策略回测完成: 超额收益率=0.5687, 策略收益率=0.0073, 股票收益率=-0.5614
股票 301353 策略回测完成: 超额收益率=0.2098, 策略收益率=-0.0594, 股票收益率=-0.2692
股票 301355 策略回测完成: 超额收益率=0.4218, 策略收益率=0.1929, 股票收益率=-0.2290
股票 301356 策略回测完成: 超额收益率=0.3598, 策略收益率=0.0020, 股票收益率=-0.3578
股票 301357 策略回测完成: 超额收益率=-0.01

D:\anaconda3\Lib\site-packages\jupyter_client\session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


股票 301503 策略回测完成: 超额收益率=0.1189, 策略收益率=-0.0136, 股票收益率=-0.1326
股票 301505 策略回测完成: 超额收益率=0.1048, 策略收益率=-0.4197, 股票收益率=-0.5245
股票 301507 策略回测完成: 超额收益率=0.2247, 策略收益率=-0.1361, 股票收益率=-0.3608
股票 301508 策略回测完成: 超额收益率=-0.0229, 策略收益率=0.0158, 股票收益率=0.0386
股票 301509 策略回测完成: 超额收益率=0.1518, 策略收益率=-0.3532, 股票收益率=-0.5049
股票 301510 策略回测完成: 超额收益率=0.3709, 策略收益率=-0.0285, 股票收益率=-0.3994
股票 301511 策略回测完成: 超额收益率=0.6733, 策略收益率=0.2429, 股票收益率=-0.4304
股票 301512 策略回测完成: 超额收益率=0.3877, 策略收益率=-0.0277, 股票收益率=-0.4155
股票 301515 策略回测完成: 超额收益率=0.3615, 策略收益率=-0.1456, 股票收益率=-0.5071
股票 301516 策略回测完成: 超额收益率=-0.1834, 策略收益率=-0.2006, 股票收益率=-0.0172
股票 301517 策略回测完成: 超额收益率=0.2410, 策略收益率=0.0778, 股票收益率=-0.1632
股票 301518 策略回测完成: 超额收益率=0.0781, 策略收益率=-0.3388, 股票收益率=-0.4170
股票 301519 策略回测完成: 超额收益率=0.1340, 策略收益率=-0.4202, 股票收益率=-0.5541
股票 301520 策略回测完成: 超额收益率=0.1762, 策略收益率=-0.1820, 股票收益率=-0.3582
错误: 股票 301522 数据不足，无法进行回测
股票 301525 策略回测完成: 超额收益率=0.4371, 策略收益率=0.0058, 股票收益率=-0.4313
股票 301526 策略回测完成: 超额收益率=0.0779, 策略收益率=-0.2677, 股票收益率=-0.3456
股

,代码,名称,超额收益率,策略收益率,股票收益率
0,600000,浦发银行,-0.06165,0.25434,0.31599
1,600004,白云机场,0.15008,-0.12814,-0.27822
2,600006,东风股份,-0.11928,0.12474,0.24402
3,600007,中国国贸,-0.01141,0.47933,0.49074
4,600008,首创环保,-0.18210,-0.00715,0.17495
5,600009,上海机场,0.15673,-0.19593,-0.35266
6,600010,包钢股份,-0.14987,-0.18119,-0.03132
7,600011,华能国际,-0.12890,-0.20492,-0.07602
8,600012,皖通高速,-0.65026,0.33331,0.98357
9,600015,华夏银行,-0.31429,0.10508,0.41937


#